<h1>
    <center>
        TMA4320 Prosjekt 2

Formålet med prosjektet er å gi en introduksjon til maskinlæring, nærmere bestemt dyplæring og dype nevrale nettverk, ved egen implementering av algoritmene. 
Ideen er å trene dataprogrammet til å gjenkjenne bildeelementer i to ulike testproblemer. I det første testproblemet er problemstillingen forenklet til gjenkjenning av fargede datapunkt, mens i det andre testproblemet skal programmet kunne skille mellom to ulike håndskrevne sifre. 
Denne rapporten skal først presentere den grunneleggende algoritmen som benyttes i treningsfasen og hovedsaklig er lik for begge problemstillingene, før hver problemstilling blir framstilt i detalj.
Valg av startparametre skal begrunnes og parametrene som ble funnet i treningsfasen skal presenteres. Deretter skal modellen med disse fikserte parametrene kjøres med nye data og resultatene bli lagt fram og diskutert.

In [ ]:
import numpy as np
import math
import random 

In [ ]:
#Lager w_k og b_k

def W(d=2):
    w_k=np.random.randn(d,d)
    return w_k

def b(d=2):
    b_k=np.random.randn(d)
    return b_k
print(W(2))
print(b(2))

In [ ]:
h = 1
#Finner Y_k+1
#Trenger først sigma, aktiveringsfunksjonen
def sigma(x):
    #return(np.tanh(x))
    #Dersom vi heller vil bruke max(), nå må x kun være vektor, denne må utvides:
    if isinstance(x, (np.ndarray, list)):
        returliste = []
        for element in x:
            returliste.append(max(element, 0))
        return(np.array(returliste))
    else:
        return(max(0, x))
    
def checkArray(array, string):
    if isinstance(array, list):
        return(np.array(array))
    elif not isinstance(array, np.ndarray):
        raise ValueError(string, "was not an np.ndarray.")
    
def findNextYVec(Y_k, W_k, b_k):
    #Denne funksjonen skal finne Y_k+1 = Y_k + h sigma(W_k * Y_k + b_k)
    #Y er en matrise (nxn), W er en matrise (nxn) og b er en vektor
    #Her er sigma en skalar funksjon som anvendes komponentvis på vektoren.
    #All input skal være arrays, men b_k vil være den eneste som lager krøll
    checkArray(b_k, 'b_k')
    # Finner produktet W_k * Y_k og legger til b_k
    vec_for_sigma = np.dot(W_k, Y_k) + b_k
    #Anvender sigma elementvis
    sigma_vec = sigma(vec_for_sigma)
    #Avkommenter for å sjekke at sigma gjør som den skal
    #print(vec_for_sigma)
    #print(sigma_vec)
    return(Y_k+h*sigma_vec)

def findNextYMat(Y_k_mat, W_k, b_k):
    #Shape[1] finner antall kolonner
    #Shape[0] finner antall rader, dvs. samme som len()
    numberOfColumns = Y_k_mat.shape[1]
    nextYMat = np.array([np.array([0.0 for i in range(numberOfColumns)]) for e in range(Y_k_mat.shape[0])])
    for column in range(numberOfColumns):
        Y_k = Y_k_mat[:, column]
        nextYMat[:, column] = findNextYVec(Y_k, W_k, b_k)
    return(np.array(nextYMat))


#Legger ved matrisene jeg har brukt for å sjekke at alt fungerer som det skal
A = np.array([[1, 2], [2, 1]])
Y_k_mat = np.array([np.array([-2, 1, 1]), np.array([1, 1, 1])])
print(findNextYMat(Y_k_mat, A, np.array([1, 1])))
#Her er altså b_k = [1, 1]


In [ ]:
#Lager Z
def n(x):
    return (np.exp(x))/(np.exp(x)+1)

def Z(Y_k, w, u, n):
    vec=Y_k.transpose()@w + u*np.ones(((Y_k.transpose()@w).size), dtype=int)
    return n(vec)

print(Z(Y_k,w,u,n))

In [ ]:
#lager J

def J(Z,c):
    return 0.5*np.linalg.norm(Z-c)**2


In [ ]:
#lager neste U, dvs. parametrene

def U_jPLussEn(Uj,tau,gradJ):
    Uj = Uj - tau*gradJ
    return Uj

In [ ]:
#adam descent
#litt usikker på hvordan disse ser ut i praksis da/hvorfor det fungerer

def adam(m,v,j,gradJ,Uj):
    beta1 = 0.9
    beta2 = 0.999
    alfa = 0.01
    epsilon = 1E-8
    g = gradJ

    m = m + (1-beta1)*g
    v = v + (1-beta2)*(g*g)

    mj = m/(1-beta1**j)
    vj = v/(1-beta2**j)

    Uj = Uj - alfa*(mj/(np.sqrt(vj)+epsilon))

    return Uj, m,v

## Inkluderte funksjoner

In [ ]:
def get_data_spiral_2d(n_samples=200):
    '''Create "observations" for training.
    
    Creates n_samples dots on two intertwined spirals.
    The dots are classified based on which spiral they belong to and
    then fuzzed by perturbing the coordinates and permuting the ordering.'''
    m1 = math.ceil(n_samples / 2)
    m2 = n_samples - m1
    
    n_turns = 1.0
    
    phi1 = np.pi
    d1 = make_spiral(m1, phi1, n_turns)

    
    phi2 = (phi1 + np.pi) % (2.0 * np.pi)
    d2 = make_spiral(m2, phi2, n_turns)
    
    features = np.hstack((d1, d2))
    labels = np.ones((n_samples,1), dtype='bool_')
    labels[m1:] = False
    
    features = features + .05 * np.random.randn(*features.shape)
    
    indexes = np.random.permutation(n_samples)
    features = features[:, indexes]
    labels = labels[indexes]


    return features, labels


def make_spiral(m, phi, n_turns):
    '''Makes points on a spiral
    
    This is a utility function for get_data_spiral_2d'''
    r = np.linspace(0.1, 1.0, m)
    a = np.linspace(0.1, 2.0 * np.pi * n_turns, m)
    xs = r * np.cos(a + phi)
    ys = r * np.sin(a + phi)
    return np.stack([xs, ys])
